In [406]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# VERİYİ MODELE HAZIRLAMA VE GEREKLİ KÜTÜPHANELERİN YÜKLENMESİ

In [407]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import cv2
import os

In [410]:
data='/gdrive/My Drive/Pneumonia/X-Ray/'


In [411]:
os.listdir(data)

['Viral', 'Bacterial']

In [412]:
CATEGORIES = ['Viral', 'Bacterial']
IMG_SIZE = 100

In [ ]:
X = []
y = []


def load_data():
    for category in CATEGORIES:
        path = os.path.join(data, category)
        class_num = CATEGORIES.index(category)

        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                X.append(new_array)
                y.append(class_num)
            except Exception as e:
                pass

load_data()

In [ ]:
X = np.array(X)
y = np.array(y)

1. ELEMENTE GÖRE TRAIN TEST VALIDATION DEĞELERİNİ AYIRDIM.

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

3. ELEMENTE GÖRE TRAIN TEST VALIDATION DEĞELERİNİ AYIRDIM.

 AŞAĞIDAKİ İŞLEMLERİ HANGİSİNDE YAPMAK İSTİYORSANIZ ONU ÇALIŞTIRIN.

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [ ]:

x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)


In [ ]:
print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)
print("x_val shape: ", x_val.shape)
print("\ny_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)
print("y_val shape: ", y_val.shape)

In [ ]:
import tensorflow as tf

train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=2)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=2)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=2)

# CNN MODELİ

In [ ]:
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(64, (3,3), padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Conv2D(128, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
model.summary()

In [ ]:
from tensorflow import keras

callback_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor = 'val_accuracy', save_best_only=True, verbose=3
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=3)
]

In [ ]:
import time
start = time.time()

history = model.fit(x_train, train_yCl,
                    batch_size=64,
                    validation_data = (x_val, valid_yCl),
                    callbacks = callback_list,
                    epochs = 1
                    #epochs = 50
                    )
print("Eğitim süresi: ", time.time() - start, "saniye")

In [ ]:
score_valid = model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test = model.evaluate(x_test, test_yCl)
print("Test Accuracy: ", score_test[1])

score_train = model.evaluate(x_train, train_yCl)
print("Train Accuracy: ", score_train[1])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.around(cm, decimals=2)
        cm[np.isnan(cm)] = 0.0
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
labels_names = [0,1]

Y_pred = model.predict(x_test)

y_pred_model = np.argmax(Y_pred, axis=1)

cm_model = confusion_matrix(y_test, y_pred_model)
plot_confusion_matrix(cm_model, CATEGORIES, normalize=False, title='Karmaşıklık Matrisi (Confusion Matrix)')

In [ ]:
from sklearn.utils.extmath import weighted_mode
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

accuracy = accuracy_score(y_test, y_pred_model)
print('CNN Accuracy Değeri: %f' % accuracy)

precision = precision_score(y_test, y_pred_model,average='weighted',zero_division=0)
print('CNN Precision Değeri: %f' % precision)

recall = recall_score(y_test, y_pred_model,average='weighted',zero_division=0)
print('CNN Recall Değeri: %f' % recall)

f1 = f1_score(y_test, y_pred_model,average='weighted',zero_division=0)
print('CNN F1 Değeri: %f' % f1)

In [ ]:
print(classification_report(y_test,y_pred_model,zero_division=0 ))


# RESNET50V2

In [ ]:
res = tf.keras.applications.ResNet50V2(
    input_shape=(100, 100, 3),
    include_top = False
)

res.trainable = False

In [ ]:
res_model = Sequential()

res_model.add(res)
res_model.add(Dropout(0.25))
res_model.add(MaxPooling2D())
res_model.add(Flatten())
res_model.add(Dense(256, activation='relu'))
res_model.add(BatchNormalization())
res_model.add(Dropout(0.5))
res_model.add(Dense(2, activation='softmax'))

In [ ]:
res_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
res_model.summary()

In [ ]:
import time
start = time.time()

hist = res_model.fit(x_train, train_yCl,
                     batch_size=128,
                     epochs = 1,
                     validation_data = (x_val, valid_yCl)
                     )

print("Eğitim süresi: ", time.time() - start, "saniye")

In [ ]:
score_train_RESNET = res_model.evaluate(x_train, train_yCl)
print("Train Accuracy: ", score_train_RESNET[1])

score_test_RESNET = res_model.evaluate(x_test, test_yCl)
print("Test Accuracy: ", score_test_RESNET[1])

score_val_RESNET = res_model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_val_RESNET[1])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import itertools
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.around(cm, decimals=2)
        cm[np.isnan(cm)] = 0.0
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:

labels_names = [0,1]

Y_pred = res_model.predict(x_test)

y_pred_resnet = np.argmax(Y_pred, axis=1)

cm_RESNET = confusion_matrix(y_test, y_pred_resnet)
plot_confusion_matrix(cm_RESNET, CATEGORIES, normalize=False, title='Karmaşıklık Matrisi (Confusion Matrix)')

In [ ]:
from sklearn.utils.extmath import weighted_mode
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

accuracy = accuracy_score(y_test, y_pred_resnet)
print(' RESNET50 Accuracy Değeri: %f' % accuracy)

precision = precision_score(y_test, y_pred_resnet,average='weighted',zero_division=0)
print('RESNET50 Precision Değeri: %f' % precision)

recall = recall_score(y_test, y_pred_resnet,average='weighted',zero_division=0)
print('RESNET50 Recall Değeri: %f' % recall)

f1 = f1_score(y_test, y_pred_resnet,average='weighted',zero_division=0)
print('RESNET50 F1 Değeri: %f' % f1)

In [ ]:
print(classification_report(y_test,y_pred_resnet,zero_division=0 ))

# INCEPTIONv3

In [ ]:
inceptionv3 = tf.keras.applications.InceptionV3(
        input_shape=(100, 100, 3),
        include_top = False,
        weights="imagenet"
)

inceptionv3.trainable = False

In [ ]:
inceptionv3_model = Sequential()

inceptionv3_model.add(inceptionv3)
inceptionv3_model.add(Dropout(0.25))
#inceptionv3_model.add(MaxPooling2D())
inceptionv3_model.add(Flatten())
inceptionv3_model.add(Dense(256, activation='relu'))
inceptionv3_model.add(BatchNormalization())
inceptionv3_model.add(Dropout(0.5))
inceptionv3_model.add(Dense(2, activation='softmax'))

In [ ]:
inceptionv3_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
import time
start = time.time()

hist = inceptionv3_model.fit(x_train, train_yCl,
                     batch_size=128,
                     #epochs = 50,
                             epochs = 1,
                     validation_data = (x_val, valid_yCl)
                     )

print("Eğitim süresi: ", time.time() - start, "saniye")

In [ ]:
score_train_INCEPV3 = inceptionv3_model.evaluate(x_train, train_yCl)
print("Train Accuracy: ", score_train_INCEPV3[1])

score_test_INCEPV3 = inceptionv3_model.evaluate(x_test, test_yCl)
print("Test Accuracy: ", score_test_INCEPV3[1])

score_val_INCEPV3 = inceptionv3_model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_val_INCEPV3[1])

In [ ]:
labels_names = [0,1]

Y_pred = inceptionv3_model.predict(x_test)

y_pred_INCEPV3 = np.argmax(Y_pred, axis=1)

cm_INCEPV3 = confusion_matrix(y_test, y_pred_INCEPV3)
plot_confusion_matrix(cm_INCEPV3, CATEGORIES, normalize=False, title='Karmaşıklık Matrisi (Confusion Matrix)')

In [ ]:
from sklearn.utils.extmath import weighted_mode
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

accuracy = accuracy_score(y_test, y_pred_INCEPV3)
print('INCEPTIONv3 Accuracy Değeri: %f' % accuracy)

precision = precision_score(y_test, y_pred_INCEPV3,average='weighted',zero_division=0)
print('INCEPTIONv3 Precision Değeri: %f' % precision)

recall = recall_score(y_test, y_pred_INCEPV3,average='weighted',zero_division=0)
print('INCEPTIONv3 Recall Değeri: %f' % recall)

f1 = f1_score(y_test, y_pred_INCEPV3,average='weighted',zero_division=0)
print('INCEPTIONv3 F1 Değeri: %f' % f1)

In [ ]:
print(classification_report(y_test,y_pred_INCEPV3 ,zero_division=0))

# Yapay Sinir Ağları

In [ ]:
from tensorflow.keras import layers

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
ann_model = Sequential()
ann_model.add(Flatten(input_shape=(100, 100, 3)))
ann_model.add(layers.Dense(16, activation="relu"))
ann_model.add(layers.Dense(8, activation="relu"))
ann_model.add(BatchNormalization())
ann_model.add(Dense(2))
ann_model.add(Activation('softmax'))
ann_model.summary()
ann_model.compile(optimizer='Adam',loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
from tensorflow import keras

callback_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor = 'val_accuracy', save_best_only=True, verbose=3
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=3)
]

In [ ]:
history = ann_model.fit(x_train, train_yCl,
                    batch_size=128,
                    validation_data = (x_val, valid_yCl),
                    callbacks = callback_list,
                    epochs = 20
                    )


In [ ]:
score_valid_ann = ann_model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test_ann = ann_model.evaluate(x_test, test_yCl)
print("Validation Accuracy: ", score_test[1])

score_train_ann = ann_model.evaluate(x_train, train_yCl)
print("Validation Accuracy: ", score_train[1])

In [ ]:
labels_names = [0,1]

Y_pred = ann_model.predict(x_test)

y_pred_ann = np.argmax(Y_pred, axis=1)

cm_ann = confusion_matrix(y_test, y_pred_ann)
plot_confusion_matrix(cm_ann, CATEGORIES, normalize=False, title='Karmaşıklık Matrisi (Confusion Matrix)')

In [ ]:
from sklearn.utils.extmath import weighted_mode
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

accuracy = accuracy_score(y_test, y_pred_ann)
print('INCEPTIONv3 Accuracy Değeri: %f' % accuracy)

precision = precision_score(y_test, y_pred_ann,average='weighted',zero_division=0)
print('INCEPTIONv3 Precision Değeri: %f' % precision)

recall = recall_score(y_test, y_pred_ann,average='weighted',zero_division=0)
print('INCEPTIONv3 Recall Değeri: %f' % recall)

f1 = f1_score(y_test, y_pred_ann,average='weighted',zero_division=0)
print('INCEPTIONv3 F1 Değeri: %f' % f1)
print("------------------------------------------------------------------")

print(classification_report(y_test,y_pred_ann ,zero_division=0))

print("------------------------------------------------------------------")
